In [5]:
%matplotlib ipympl
# %matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from utils import create_session_config
import tensorflow_probability as tp
import numpy as np
from models.utils import covariance,correlation_matrix,tf_cov,assert_cov_positive_definite,corr,print_tensor,print_tensors,constrain_cdf
from models.nn_pdf_common import transform_x,density_estimator,create_pdf_layer_mv,create_partially_monotone_dense_layer,\
    create_monotone_dense_layer,create_positive_weights,create_bias
from matplotlib import pyplot as plt
from scipy.stats import multivariate_normal
from flags import FLAGS
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import os
np.set_printoptions(threshold=np.inf)
from data.registry import inv_sin
from models.utils import get_train_inputs
from tensorflow.python import debug as tf_debug
import itertools
import tensorflow as tf
import tensorflow_probability as tp
from models.utils import log_likelihood_from_cdfs_transforms, constrain_cdf, metric_loglikelihood, train_op, \
    print_tensor, add_all_summary_stats, extract_xy

In [4]:
FLAGS.dir = '/home/pawel/PycharmProjects/RM_labs/tmp2/'
os.chdir(os.path.join(FLAGS.dir,'inv_sin'))
FLAGS.data_set="inv_sin"
data_loader = inv_sin() 

loaded data: inv_sin.npz


In [6]:
def get_mixtures(output, num_marginals):
    marginal_params = tf.split(output, num_or_size_splits=num_marginals, axis=1)
    mixtures = []

    for marginal_id in range(num_marginals):
        with tf.variable_scope("mixture_%d"%marginal_id):
            out_logits, out_sigma, out_mu = tf.split(marginal_params[marginal_id], num_or_size_splits=3, axis=1)
            out_sigma = print_tensor(add_all_summary_stats(tf.maximum(1e-24,tf.square(out_sigma), name="sigma")))

            mix = tp.distributions.MixtureSameFamily(
                mixture_distribution=tp.distributions.Categorical(logits=out_logits),
                components_distribution=tp.distributions.Normal(loc=out_mu, scale=out_sigma))

            mixtures.append(mix)
    return mixtures

def get_loss(log_likelihood):
    return tf.negative(tf.reduce_mean(log_likelihood), name="loss")

def generate_ensemble(mix):
    return tf.reshape(mix.sample(1, name="sample"), [-1,1])

def compute_log_prob_mixture(mix, y, i):
    with tf.variable_scope("log_pdf_%d"%i):
        return tf.reshape(mix.log_prob(tf.reshape(y, [-1])),[-1,1])

def log_likelihood(cov_type, mixtures, y, lls,x, params):
    cdfs = []
    for i, mix in enumerate(mixtures):
        # this has to be 1-D so mixture computes parametrization per data point
        y_component = tf.reshape(tf.slice(y, [0, i], size=[-1, 1]), [-1], name="y_component%d"%i)
        cdfs.append(constrain_cdf(tf.reshape(mix.cdf(y_component, name="component_%d_cdf"%i),[-1,1])))

    return log_likelihood_from_cdfs_transforms(cov_type, cdfs, lls, x, params)

def compute_lls(mixtures, y):
    return [compute_log_prob_mixture(mix, tf.slice(y, [0, i], size=[-1, 1]), i) for i, mix in enumerate(mixtures)]

In [12]:
tf.reset_default_graph()
tf.set_random_seed(10)

ds = get_train_inputs(data_loader, batch_size=200)
iter = tf.data.Iterator.from_structure(ds.output_types,ds.output_shapes)
training_init_op = iter.make_initializer(ds)
batch=iter.get_next()

global_step_tensor = tf.Variable(0, trainable=False, name='global_step')
y_size = 1

y=batch['y']
x=batch['x']
x_size=1

mode=tf.estimator.ModeKeys.TRAIN
params={}
params['hidden_units'] = 10
params['learning_rate'] = 0.001
params['num_layers'] = 2
params['k_mix'] = 10
params['cov_type'] = 'const'

k_mix = params["k_mix"]
num_layers = params["num_layers"]
hidden_units = params['hidden_units']
learning_rate = params["learning_rate"]
cov_type = params["cov_type"]

layer = x

n_out = k_mix * 3 * y_size  # pi, mu, stdev

if x is not None:
    for i in range(num_layers):
        layer = tf.layers.dense(layer, units=hidden_units, activation=tf.nn.tanh, name="h_layer_%d" % i)

    Wo = tf.get_variable("Wo", shape=[params['hidden_units'], n_out], dtype=tf.float32)
    bo = tf.get_variable("bo", shape=[1, n_out], dtype=tf.float32)

    output = tf.matmul(layer, Wo) + bo
else:
    output = tf.get_variable("output", shape=(1, n_out))

mixtures = get_mixtures(output, y_size)

marginal_lls = compute_lls(mixtures, y)
ll = log_likelihood(cov_type, mixtures, y, marginal_lls, x, params)
loss = get_loss(ll)

optimizer = tf.train.AdamOptimizer(learning_rate=params['learning_rate'])

train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())

def train_loop(iters, tain_op):
    session.run(training_init_op)
    
    print("step, loss: ",session.run([tf.train.get_global_step(),loss]))
  
    for step in range(iters):  
        step_val,loss_val,_=session.run([tf.train.get_global_step(),loss,tain_op])
        if step % 1000 ==0:
            print("step: {step}, loss: {loss}".format(step=step_val,loss=loss_val))
        
    print("loss: ",session.run(loss))

with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.04))) as session:
#     session = tf_debug.LocalCLIDebugWrapperSession(session)
#     session.add_tensor_filter("has_inf_or_nan", tf_debug.has_inf_or_nan)
    session.run(tf.global_variables_initializer())  
    train_loop(10000000, train_op)


step, loss:  [0, 13.745109]
step: 1, loss: 14.099165916442871
step: 1001, loss: 0.1770516186952591
step: 2001, loss: 0.0430937260389328
step: 3001, loss: 0.026956617832183838
step: 4001, loss: -0.023929934948682785
step: 5001, loss: -0.026690278202295303
step: 6001, loss: -0.21177475154399872
step: 7001, loss: -0.11148286610841751
step: 8001, loss: -0.0903037041425705
step: 9001, loss: -0.13646581768989563
step: 10001, loss: -0.1312563121318817
step: 11001, loss: -0.1992224156856537
step: 12001, loss: -0.16778677701950073
step: 13001, loss: -0.13733763992786407
step: 14001, loss: -0.08363212645053864
step: 15001, loss: -0.10829459875822067
step: 16001, loss: -0.16952566802501678
step: 17001, loss: -0.12203441560268402
step: 18001, loss: -0.20326006412506104
step: 19001, loss: -0.047660037875175476
step: 20001, loss: -0.07615961134433746
step: 21001, loss: -0.13758069276809692
step: 22001, loss: -0.10781072825193405
step: 23001, loss: -0.1789550930261612
step: 24001, loss: -0.1532120853

Exception ignored in: <bound method TF_Output.<lambda> of <tensorflow.python.pywrap_tensorflow_internal.TF_Output; proxy of <Swig Object of type 'TF_Output *' at 0x7f8d4c651fc0> >>
Traceback (most recent call last):
  File "/home/pawel/tensorflow/lib/python3.5/site-packages/tensorflow/python/pywrap_tensorflow_internal.py", line 963, in <lambda>
    __del__ = lambda self: None
KeyboardInterrupt


step: 30001, loss: -0.12085758149623871
step: 31001, loss: -0.20843103528022766
step: 32001, loss: -0.22926589846611023
step: 33001, loss: -0.20736920833587646
step: 34001, loss: -0.1003480926156044
step: 35001, loss: -0.17286212742328644
step: 36001, loss: -0.22291794419288635
step: 37001, loss: -0.22853723168373108
step: 38001, loss: -0.2658086121082306
step: 39001, loss: -0.1630910038948059
step: 40001, loss: -0.06404709815979004
step: 41001, loss: -0.19753113389015198
step: 42001, loss: -0.07987198233604431
step: 43001, loss: -0.1788136512041092
step: 44001, loss: -0.3183062672615051
step: 45001, loss: -0.15315082669258118
step: 46001, loss: -0.3259912133216858
step: 47001, loss: -0.15781714022159576
step: 48001, loss: -0.23492465913295746
step: 49001, loss: -0.22411257028579712
step: 50001, loss: -0.115387462079525
step: 51001, loss: -0.15677422285079956
step: 52001, loss: -0.18423543870449066
step: 53001, loss: -0.2031368613243103
step: 54001, loss: -0.2183283269405365
step: 5500

KeyboardInterrupt: 